# Intermediate OpenDP Data Analysis


The majority of the library lies behind "contrib", a feature-gated, opt-in flag. 
Each component in the library must pass a vetting process before it is moved out of "contrib".
I am opting into "contrib" because the analysis in this notebook will need to use features that have not passed the vetting process.

In [1]:
from opendp.mod import enable_features
enable_features('contrib')

### Transformations Intuition

The bulk of the library interface consists of constructors. One such example is a constructor for a clamp transformation:

In [34]:
from opendp.trans import make_clamp
clamper = make_clamp(bounds=(0, 10))

clamper([-1, 0, 3, 5, 10, 20])

[0, 0, 3, 5, 10, 10]

In [3]:
clamper.check(1, 1)

True

We use chaining to combine multiple transformations:

In [4]:
from opendp.trans import make_cast_default

caster = make_cast_default(TIA=str, TOA=int)

preprocessor = caster >> clamper
preprocessor(["1", "2", "3", "20", "a"])

[1, 2, 3, 10, 0]

In [5]:
preprocessor.check(1, 1)

True

The input metric and output metric are implicitly `SymmetricDistance`.

Lets start a more fleshed out analysis of the PUMS dataset.

Let's examine how we can process csv data with Transformations.

In [6]:
# establish public information
col_names = ["state", "puma", "sex", "age", "educ", "income", "latino", "black", "asian", "married"]
# the greatest number of records that any one individual can influence in the dataset
max_influence = 1
# we can also reasonably intuit that age and income will be numeric,
#     as well as bounds for them, without looking at the data
age_bounds = (0, 100)
income_bounds = (0, 500_000)

In [7]:
import os
data_path = os.path.join('.', 'PUMSExtract1000.csv')

with open(data_path) as input_file:
    data = input_file.read()

print('\n'.join(data.split('\n')[:6]))

833320,6,64900,0,45,6,6000,0,0,0,1
875563,6,66117,1,41,8,13000,1,0,0,1
431101,6,62802,0,63,14,17810,0,0,0,1
941157,6,66123,1,71,15,3600,0,0,1,1
35464,6,60600,0,44,5,10000,0,0,0,0
451867,6,63301,1,49,1,0,0,0,1,0


As we can see from the first few rows, it is intentional that there are no column names in the data.
If your data has column names, you will want to strip them out before passing data into your function.

I'm going to pull a few constructors from the [Dataframes section in the user guide](https://docs.opendp.org/en/stable/user/transformation-constructors.html#dataframe).

We start with `make_split_dataframe` to parse one large string containing all the csv data into a dataframe.
`make_split_dataframe` expects us to pass column names, which we can grab out of the public information.
`make_select_column` will then index into the dataframe to pull out a column where the elements have a given type `TOA`.
The `TOA` argument won't cast for you; the casting comes later!

Now if you run the transformation on the data, you will get a list of incomes as strings.
I've limited the output to just the first few income values.

In [8]:
from opendp.trans import make_split_dataframe, make_select_column
income_preprocessor = (
    # Convert data into a dataframe where columns are of type Vec<str>
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="income", TOA=str)
)

In [9]:
transformed = income_preprocessor(data)
print(type(transformed))
print(transformed[:6])

<class 'list'>
['6', '8', '14', '15', '5', '1']


### Casting
Income doesn't make sense as a string for our purposes,
so we can just extend the previous preprocessor to also cast and impute.

In [10]:
from opendp.trans import make_cast, make_impute_constant

# make a transformation that casts from a vector of strings to a vector of ints
cast_str_int = (
    # Cast Vec<str> to Vec<Option<int>>
    make_cast(TIA=str, TOA=int) >>
    # Replace any elements that failed to parse with 0, emitting a Vec<int>
    make_impute_constant(0)
)

# replace the previous preprocessor: extend it with the caster
income_preprocessor = income_preprocessor >> cast_str_int
print(income_preprocessor(data)[:6])

[6, 8, 14, 15, 5, 1]


Great! We've successfully transformed CSV data into an integer vector from the income column.

### Transformation Structure
The general approach is that we can build up lengthy computation chains out of isolated transformations.
Each constituent transformation represents an isolated unit of computation with provable stability properties.

The following snip shows the definition of a Transformation in the core rust library.
```rust
pub struct Transformation<DI: Domain, DO: Domain, MI: Metric, MO: Metric> {
    pub input_domain: DI,
    pub output_domain: DO,
    pub function: Function<DI, DO>,
    pub input_metric: MI,
    pub output_metric: MO,
    pub stability_relation: StabilityRelation<MI, MO>,
}
```

Lets explain each of the struct members.
```rust
    ...
    pub input_domain: DI,
    pub output_domain: DO,
    ...
```
The input and output domain strictly defines permissible input and output values.
When you attempt to chain any two transformations, the output domain of the first transformation must match the input domain of the second transformation.
The resulting chained transformation contains the input domain of the first transformation, the output domain of the second transformation, as well as the two functions composed.

```rust
    ...
    pub function: Function<DI, DO>,
    ...
```
When we invoked the following transformation, the python data structure was translated into a low-level C representation, the rust `function` was evaluated, and the result shipped back out to familiar python data structures.

In [11]:
cast_str_int(["null", "1.", "2", "456"])

[0, 0, 2, 456]

We also have input and output metrics.
```rust
    ...
    pub input_metric: MI,
    pub output_metric: MO,
    ...
```
Examples of metrics are `HammingDistance`, `SymmetricDistance`, `AbsoluteDistance` and `L1Distance`. 
They behave in the same way that the input and output domains do when chaining.

```rust
    ...
    pub stability_relation: StabilityRelation<MI, MO>,
    ...
```
Finally, the stability relation. 
It is a function that takes in an input and output distance, in the respective metric spaces, and returns a boolean.
The function relates the input and output distances, returning False if the output distance is too small with respect to the input distance.

For example, we know that the casting transformation is row-by-row, so we should expect that for any symmetric distance `a`, the pair of distances (`a`, `a`) are related.

In [12]:
a = 3
cast_str_int.check(d_in=a, d_out=a)

True

In [13]:
cast_str_int.check(d_in=a, d_out=a - 1)

False

When any two compatible transformations are chained, the resulting transformation contains a functional composition of the relations.

Ultimately, all pieces are used to construct the new transformation:

| input | chaining | output |
|---:|:---:|:---|
| input_domain_1 | output_domain_1 == input_domain_2 | output_domain_2 |
| function_1 |composed with| function_2 |
| input_metric_1 | output_metric_1 == input_metric_2 | output_metric_2 |
| stability_relation_1 | composed with | stability_relation_2 |

As you've seen above, when we want to create a transformation, we use "constructor" functions. These are, by convention, prefixed with `make_`.

An example implementation of a casting transformation constructor is provided. I'll break it down into three parts.

```rust
// 1.
pub fn make_cast_default<DIA, TOA>() -> Fallible<Transformation<
    VectorDomain<AllDomain<TIA>>, VectorDomain<AllDomain<TOA>>, 
    SymmetricDistance, SymmetricDistance>>

    // 2.
    where TIA: 'static + Clone + CheckNull, 
          TOA: 'static + RoundCast<TIA> + Default + CheckNull {

    // 3.
    Ok(Transformation::new(
        VectorDomain::new(AllDomain::new()),
        VectorDomain::new(AllDomain::new()),
        Function::new(move |arg: &Vec<TIA>|
            arg.iter().map(|v| TOA::round_cast(v.clone()).unwrap_or_default()).collect()),
        SymmetricDistance::new(),
        SymmetricDistance::new(),
        StabilityRelation::new_from_constant(1)))
}
```

The first part is the function signature:
```rust
pub fn make_cast_default<TIA, TOA>() -> Fallible<Transformation<
    VectorDomain<AllDomain<TIA>>, VectorDomain<AllDomain<TOA>>, 
    SymmetricDistance, SymmetricDistance>>
    ...
```
Most of the signature consists of types. 
Rust is strictly typed, so the code needs to be very explicit about what the type of the constructor function's inputs and outputs are. 

This is a generic function with two type arguments `TIA` and `TOA`, standing for "atomic input type" and "atomic output type".
There are zero first-class arguments `()`.

The constructor returns a fallible transformation.
The last two lines specify the types of the input/output domains/metrics.

The second part is the where clause:
```rust
    ...
    where TIA: 'static + Clone + CheckNull, 
          TOA: 'static + RoundCast<TIA> + Default + CheckNull {
    ...
```
A where clause lists bounds on the acceptable types to be used in the function.
You can interpret this as, "the compiler will enforce that `TIA` must be some type that has the `Clone` and `CheckNull` traits. 
In other words, while I don't specify what `TIA` must be up-front, I can bound what type it may be to types that are cloneable and have some concept of null-checking.
`TOA`, in particular, has a `RoundCast` trait, which can be used to cast from type `TIA` to `TOA`. 

The final part is the function body, which just creates and implicitly returns a Transformation struct.
```rust
    ...
    Ok(Transformation::new(
        VectorDomain::new(AllDomain::new()),
        VectorDomain::new(AllDomain::new()),
        Function::new(move |arg: &Vec<TIA>|
            arg.iter().map(|v| TOA::round_cast(v.clone()).unwrap_or_default()).collect()),
        SymmetricDistance::new(),
        SymmetricDistance::new(),
        StabilityRelation::new_from_constant(1)))
}
```
Each argument corresponds to a struct member.
We take advantage of a handy syntax for creating un-named functions:
In the example function addition function, `|a, b| a + b`. takes two arguments, `a` and `b`. The function body is `a + b`.

With this shorthand in-hand, we create a function that casts the data by iterating over each record `v`, casting, and replacing nulls with the default value for the type.

We also take advantage of a convenient constructor for building `c`-stable relations.
Since the cast function is row-by-row, it is 1-stable.

### Private Count
Time to compute our first aggregate statistic.
Suppose we want to know the number of records in the dataset.

We can use the [list of aggregators](https://docs.opendp.org/en/stable/user/transformation-constructors.html#aggregators)
in the Transformation Constructors section of the user guide to find `make_count`.

In [14]:
from opendp.trans import make_count
count = income_preprocessor >> make_count(TIA=int)
# NOT a DP release!
count_response = count(data)

Be careful!
`count` is still only a transformation,
so the output in `count_response` is not a differentially private release.
You will need to chain with a measurement to create a differentially private release.

We use `make_base_geometric` below, because `make_base_geometric` has an integer support.
Notice that we now import from `opendp.meas`, and the resulting `type(dp_count)` is `Measurement`. This tells us that the output will be a differentially private release.

In [15]:
from opendp.meas import make_base_geometric
dp_count = count >> make_base_geometric(scale=1.)

In any realistic situation, you would likely want to estimate the budget utilization before you make a release.
Use a search utility to quantify the privacy expenditure of this release.

In [16]:
from opendp.mod import binary_search

# estimate the budget...
epsilon = binary_search(
    lambda eps: dp_count.check(d_in=max_influence, d_out=eps),
    bounds=(0., 100.))
print("DP count budget:", epsilon)

# ...and then release
count_release = dp_count(data)
print("DP count:", count_release)

DP count budget: 1.0
DP count: 10001


### Measurement Structure

Measurements are very similar to Transformations, with two key differences.

```rust
pub struct Measurement<DI: Domain, DO: Domain, MI: Metric, MO: Measure> {
    pub input_domain: DI,
    pub output_domain: DO,
    pub function: Function<DI, DO>,
    pub input_metric: MI,
    pub output_measure: MO,
    pub privacy_relation: PrivacyRelation<MI, MO>,
}
```

First, the `output_metric` is replaced with an `output_measure`, as distances in the output space are measured in terms of divergences between probability distributions.

Second, the name of the relation has changed from a stability relation to a privacy relation. 
This is because the relation between distances now carries meaning with respect to privacy.

### Private Sum

Suppose we want to know the total income of our dataset.
First, take a look at [the list of aggregators](https://docs.opendp.org/en/stable/user/transformation-constructors.html#aggregators).
`make_bounded_sum` seems to meet our requirements.
As indicated by the function's API documentation, it expects bounded data,
so we'll also need to chain the transformation from `make_clamp` with the `income_preprocessor`.

In [17]:
from opendp.trans import make_clamp, make_bounded_sum
bounded_income_sum = (
    income_preprocessor >>
    # Clamp income values
    make_clamp(bounds=income_bounds) >>
    # These bounds must be identical to the clamp bounds, otherwise chaining will fail
    make_bounded_sum(bounds=income_bounds)
)

In this example, instead of just passing a scale into `make_base_geometric`,
lets say I want whatever scale will make my measurement 1-epsilon DP.
Again, I can use a search utility to find such a scale.

In [18]:
from opendp.mod import binary_search_param

discovered_scale = binary_search_param(
    lambda s: bounded_income_sum >> make_base_geometric(scale=s),
    d_in=max_influence,
    d_out=1.)

dp_sum = bounded_income_sum >> make_base_geometric(scale=discovered_scale)


Or more succinctly...

In [19]:
from opendp.mod import binary_search_chain

dp_sum = binary_search_chain(
    lambda s: bounded_income_sum >> make_base_geometric(scale=s),
    d_in=max_influence,
    d_out=1.)

# ...and make our 1-epsilon DP release
print("DP sum:", dp_sum(data))

DP sum: 1208520


### Private Mean

We may be more interested in the mean age.
Just like before, we reference the docs to find `make_sized_bounded_mean`.
The name of the constructor indicates that it expects sized, bounded data,
and the docstring points us toward preprocessors we can use.

Sized data is data that has a known number of rows.
The constructor enforces this requirement
because knowledge of the dataset size was necessary to establish the privacy proof.

Luckily, we've already made a DP release of the number of rows in the dataset,
which we can reuse as an argument here.

Putting the previous sections together, our bounded mean age is:

In [1]:
from opendp.trans import make_cast_default, make_bounded_resize, make_sized_bounded_mean
from opendp.mod import OpenDPException

try:
    mean_age_preprocessor = (
        # Convert data into a dataframe of string columns
        make_split_dataframe(separator=",", col_names=col_names) >>
        # Selects a column of df, Vec<str>
        make_select_column(key="age", TOA=str) >>
        # Cast the column as Vec<float>, and fill nulls with the default value, 0.
        make_cast_default(TIA=str, TOA=float) >>
        # Clamp age values
        make_clamp(bounds=age_bounds)
    )
except OpenDPException as err:
    assert err.message.startswith("Intermediate domains don't match.")
    print(err.message)

NameError: name 'make_split_dataframe' is not defined

Wait a second! The intermediate domains don't match?
In this case, we casted to float-valued data, but `make_clamp` was built with integer-valued bounds,
so the clamp is expecting integer data.
Therefore, the output of the cast is not a valid input to the clamp.
We can fix this by adjusting the bounds and trying again.

In [21]:
float_age_bounds = tuple(map(float, age_bounds))

mean_age_preprocessor = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>
    # Cast the column as Vec<float>, and fill nulls with the default value, 0.
    make_cast_default(TIA=str, TOA=float) >>
    # Clamp age values
    make_clamp(bounds=float_age_bounds) >>
    # Resize the dataset to length `count_release`.
    #     If there are fewer than `count_release` rows in the data, fill with a constant of 20.
    #     If there are more than `count_release` rows in the data, only keep `count_release` rows
    make_bounded_resize(size=count_release, bounds=float_age_bounds, constant=20.) >>
    # Compute the mean
    make_sized_bounded_mean(size=count_release, bounds=float_age_bounds)
)


I stopped just short of chaining with a measurement because we're working with float data.
There are some extra considerations to take in mind with floating-point data,
which are [covered in the user guide](https://docs.opendp.org/en/latest/user/measurement-constructors.html#floating-point).

With the assumption that you understand the ramifications, I'll go ahead and finish the query.

In [22]:
from opendp.meas import make_base_laplace
enable_features("floating-point")
# add laplace noise
dp_mean = mean_age_preprocessor >> make_base_laplace(scale=1.0)

mean_release = dp_mean(data)
print("DP mean:", mean_release)

DP mean: 0.21497149083820533


Depending on your use-case, you may find greater utility separately releasing a DP sum and a DP count,
and then postprocessing them into the mean.
In the above mean example, you could even take advantage of this to avoid using floating-point numbers.


### Private Variance

This is the last quick example, just to show a complete computation chain.
In this example, I chain with the gaussian mechanism instead, with a budget of .1 epsilon, 1e-5 delta.

In [23]:
from opendp.meas import make_base_gaussian
from opendp.trans import make_sized_bounded_variance

variance = (
    # Convert data into a dataframe of string columns
    make_split_dataframe(separator=",", col_names=col_names) >>
    # Selects a column of df, Vec<str>
    make_select_column(key="age", TOA=str) >>
    # Cast the column as Vec<float>, and fill nulls with the default value, 0.
    make_cast_default(TIA=str, TOA=float) >>
    # Clamp age values
    make_clamp(bounds=float_age_bounds) >>
    # Resize the dataset to length `count_release`.
    make_bounded_resize(size=count_release, bounds=float_age_bounds, constant=20.) >>
    # Compute the variance
    make_sized_bounded_variance(size=count_release, bounds=float_age_bounds)
)

dp_variance = binary_search_chain(
    lambda s: variance >> make_base_gaussian(scale=s),
    d_in=max_influence,
    d_out=(1., 1e-5) # (epsilon, delta)-DP
)

print("DP variance:", dp_variance(data))

DP variance: -1.039669609948424


### Combinators

Combinators constructors that take other measurements or transformations as arguments.

You're already familiar with chainers, which are a type of combinator.

In [24]:
from opendp.comb import make_chain_tt

chained = make_chain_tt(clamper, caster)

chained(["1", "2", "3", "10", "a"])

[1, 2, 3, 10, 0]

Another kind of combinator is composition. We can combine multiple measurements to create one measurement that represents the composition:

In [25]:
from opendp.comb import make_basic_composition

composed = make_basic_composition(dp_sum, dp_mean)

composed(data)

'(546144, 0.2937861188251779)'

Another type of combinator is an amplifier. In this example I'll apply the amplifier to a dp variance estimator:

In [33]:
from opendp.comb import make_population_amplification
variance = make_sized_bounded_variance(size=count_release, bounds=float_age_bounds)

dp_variance = binary_search_chain(
    lambda s: variance >> make_base_laplace(scale=s),
    d_in=max_influence,
    d_out=1.
)

make_population_amplification(dp_variance, 100_000).check(1, .2)


True

You'll notice that we found a dp variance estimator that was 1 epsilon-DP, but after amplification, it is at least .05 epsilon-DP. We are taking advantage of the knowledge that the dataset was a simple sample from a larger population with at least 100,000 individuals.

For more information... 
- Docs website: https://docs.opendp.org
- Github Repo:  https://github.com/opendp/opendp


### Contrib, Vetting and Proofs

As mentioned before, much of the library is still in "contrib".
A requirement of the vetting process is having the code supported by a proof document. 
The library is designed to make this as easy as possible, because it consists of modular building blocks (Transformations and Measurements) for which encapsulated proofs may be written.

Each Transformation or Measurement proof must show the following:
1. That the function, when evaluated on any element in the input domain, emits a value in the output domain.
2. That the relation always returns false if the function is not (d_in, d_out)-close for all d_in and d_out.
3. That your choices of metrics/measures are compatible with your domains.

